In [1]:
import os
import re
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn import metrics

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [5]:
df = pd.read_csv('data/real_estate_for_mlmodels.csv')
df.head()

,City,State,Country,PropertyType,SubArea,CompanyName,TownshipSocietyName,Description,ClubHouse,School/UniversityInTownship,HospitalInTownShip,MallInTownShip,ParkJoggingTrack,SwimmingPool,Gym,PropertyAreainSqFt,PriceInLakhs,PriceBySubArea,AmenitiesScore,PriceByAmenitiesScore,Noun_Counts,Verb_Counts,Adjective_Counts,boasts elegant,elegant towers,every day,great community,mantra gold,offering bedroom,quality specification,stories offering,towers stories,world class
0,Pune,Maharashtra,India,1.0,Bavdhan,Shapoorji paloonji,Vanaha,shapoorji paloonji comunity located suburbs ba...,1,1,1,1,1,1,1,492.0,39.0,58.044000,7,72.666667,9,1,3,0,0,0,0,0,0,0,0,0,0
1,Pune,Maharashtra,India,2.0,Bavdhan,Shapoorji paloonji,Vanaha,vanaha township located near lonavala hill ran...,1,1,1,1,1,1,1,774.0,65.0,58.044000,7,72.666667,9,1,3,0,0,0,0,0,0,0,0,0,0
2,Pune,Maharashtra,India,3.0,Bavdhan,Shapoorji paloonji,Vanaha,vanaha society suitable aged group people play...,1,1,1,1,1,1,1,889.0,74.0,58.044000,7,72.666667,9,1,3,0,0,0,0,0,0,0,0,0,0
3,Pune,Maharashtra,India,3.0,Bavdhan,Shapoorji paloonji,Vanaha,vanaha township offering bhk grand prpoerties ...,1,1,1,1,1,1,1,1018.0,89.0,58.044000,7,72.666667,8,1,3,0,0,0,0,0,0,0,0,0,0
4,Pune,Maharashtra,India,2.0,Mahalunge,Godrej properties,Godrej hills retreat,area hub prestigious schools like bishop high ...,1,1,1,1,1,1,1,743.0,74.0,73.555556,7,72.666667,12,1,6,0,0,0,0,0,0,0,0,0,0
